En este archivo vamos a preprocesar el texto de manera que saquemos las etiquetas de cada tipo de palabra (POS tagging). Después pasaremos texto + etiquetas a un cierto formato, que será el que le pasemos al modelo entrenado para que resuelva las correferencias en el texto. Cabe destacar que el texto que vamos a usar está en español.

### Al terminar, sacar hacer el requisitos.txt

In [2]:
%pip install -r requirements.txt

  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
Using cached nltk-3.9.2-py3-none-any.whl (1.5 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


# 1 Sacar etiquetas
Como hemos mencionado antes, primero vamos a sacar las etiquetas de cada palabra en un texto, comprobaremos el funcionamiento de varios modelos y discutiremos con cuál nos quedamos.

## 1.1 Imports
El primer paso es definir los imports que vamos a usar durante esta parte, así como si tenemos que descargar datos para dichos modelos.

In [2]:
import spacy
import stanza
from flair.data import Sentence
from flair.models import SequenceTagger

In [3]:
spacy.cli.download('es_core_news_sm')
print('-'*80)
spacy.cli.download('es_dep_news_trf')
print('-'*80)
stanza.download('es')

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
--------------------------------------------------------------------------------
✔ Download and installation successful
You can now load the package via spacy.load('es_dep_news_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
--------------------------------------------------------------------------------


2025-12-28 16:01:09 INFO: Downloaded file to C:\Users\Jorge\stanza_resources\resources.json
2025-12-28 16:01:09 INFO: Downloading default packages for language: es (Spanish) ...
2025-12-28 16:01:10 INFO: File exists: C:\Users\Jorge\stanza_resources\es\default.zip
2025-12-28 16:01:13 INFO: Finished downloading models and saved to C:\Users\Jorge\stanza_resources


## 1.2 Corpus de prueba
Antes de pasar a la acción definimos un corpus de prueba para evaluar los modelos.En este vamos a tener todas las frases que vayamos a usar a lo largo de todo el preprocesamiento


In [27]:
corpus = [
    'El gato come pescado.',
    'El equipo de fútbol jugó su partido. Este ganó con facilidad.',
    'El coche rojo se estropeó, así que lo llevé al taller.',
    'La presidenta y el director se reunieron; ella habló primero.',
    'Entregué el informe a la jefa después de que ella lo leyera.',
    'Los equipos trabajaron duro, y al final ellos ganaron el premio.',
    'A pesar de sus problemas, el artista terminó su obra.',
    'El hermano de María dijo que él vendría.',
    'En su oficina, el abogado revisó los documentos.',
    'El libro que leí es fascinante; este autor siempre sorprende.',
    'El gato persiguió al ratón, pero este logró escapar.',
    'Hablé con Pedro sobre su proyecto y luego él me envió los archivos.',
    'Ayer hablé con Juan, le dije: "dímelo, por favor", y no me lo quiso decir',
    'Dímelo',
    'Dí me lo',
    'Di me lo',
    "La cabra de mi amigo Juan arremete todos los días contra la valla.",
    "Mi madre me dijo: 'cómete la tarta', por lo que me la comí.",
    "Mi profesor me dijo: 'comete este error y suspenderás', por lo que tuve mucho cuidado."
    "No sé si viste ayer a mi padre, pero yo sí.",
    "Vístete rápido, que ya nos vamos.",
    "Muéstrame tu chaqueta nueva."
]

## 1.3 Pruebas
Vamos a empezar a probar modelos sobre todas las frases que hemos definido antes.

### 1.3.1 Spacy - es_core_news_sm

In [28]:
nlp = spacy.load("es_core_news_sm")  # Para español
test1 = []
for text in corpus: test1.append(nlp(text))
for doc in test1:
    for token in doc:
        print(token.text, token.pos_)
    print()

El DET
gato NOUN
come VERB
pescado ADJ
. PUNCT

El DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
Este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

El DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
al ADP
taller NOUN
. PUNCT

La DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

Entregué PROPN
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

Los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

A ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

El DET
hermano NOUN
de ADP
María PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

En ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

El DET
libro NOU

En general lo hace muy bien, aunque confunde VERB que comienzan una oración con PROPN (sustantivos propios) y, en la primera oración, confunde el sustantivo pescado por el adjetivo, por lo que podemos intuir que cometerá más veces ese error. Por lo que a nuestra futura tarea respecta, el primer error puede ser garrafal, pues podría pensar el modelo que un pronombre se refiere a un verbo de esos mal etiquetados. Además, el verbo decir más los enclíticos me y lo lo considera un sustantivo, fallo enorme ya que son dos pronombres tras el verbo.
Veamos el resto de modelos.

### 1.3.2 Spacy - es_dep_news_trf

In [29]:
nlp = spacy.load("es_dep_news_trf")
test2 = []
for text in corpus: test2.append(nlp(text))
for doc in test2:
    for token in doc:
        print(token.text, token.pos_)
    print()

El DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

El DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
Este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

El DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
al ADP
taller NOUN
. PUNCT

La DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

Entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

Los DET
equipos NOUN
trabajaron VERB
duro ADV
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

A ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

El DET
hermano NOUN
de ADP
María PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

En ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

El DET
libro NOU

En este caso vemos que corrige todos los errores previos, salvo el caso de los enclíticos, que los considera verbos (bien, pero solo la raíz es el verbo). Esto muestra que este modelo es mejor, pero no perfecto, pues confunde, por ejemplo "Dí", de separar los enclíticos en "Dímelo", por AUX, pero sin la tilde dice correctamente que es un verbo, esto lo tendremos en cuenta más adelante.

### 1.3.3 Spacy - es_dep_news_trf (texto en minúsculas)

In [30]:
nlp = spacy.load("es_dep_news_trf")
test3 = []
for text in corpus: test3.append(nlp(text.lower()))
for doc in test3:
    for token in doc:
        print(token.text, token.pos_)
    print()

el DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

el DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

el DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
al ADP
taller NOUN
. PUNCT

la DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

los DET
equipos NOUN
trabajaron VERB
duro ADV
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

a ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

el DET
hermano NOUN
de ADP
maría PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

en ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

el DET
libro NOU

Al hacer varias pruebas nos damos cuenta que estos modelos son <em>case sensitive</em>, es decir, las palabras en mayúsculas y minúsculas importan. En este caso vemos que se solventa el problema del AUX en "dí", pero aún así palabras como "dímelo" no las hace bien. Probemos con otro modelo, en este caso Stanza.

### 1.3.4 Stanza

In [31]:
nlp = stanza.Pipeline('es', verbose=False)
test4 = []
for text in corpus: test4.append(nlp(text))
for doc in test4:
    for sentence in doc.sentences:
        for word in sentence.words:
            print(word.text, word.pos)
    print()

El DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

El DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
Este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

El DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
a ADP
el DET
taller NOUN
. PUNCT

La DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

Entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

Los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
a ADP
el DET
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

A ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

El DET
hermano NOUN
de ADP
María PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

En ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

El D

En este caso el modelo funciona muy similar a Spacy con es_dep_news_trf. Tampoco identifica correctamente los pronombres en "dímelo", y en este caso lo marca como interjección (INTJ). Cuando solo ponemos el verbo con enclíticos (Dímelo) lo considera verbo, y cuando lo separamos con espacios las partes analiza bien los pronombres pero el verbo vuelve a confundirlo con interjección, aunque si el verbo está sin tilde lo sigue marcanto como INTJ. Probemos con el texto en minúsculas. 

In [44]:
from itertools import product

CLITICS = ['me','te','se','lo','la','los','las','le','les','nos','os']

# construye combos posibles (hasta 2 clíticos en la práctica; hay casos de 3 pero raros)
def generate_clitic_combos():
    combos = set()
    # uno
    for c in CLITICS:
        combos.add(c)
    # dos (orden correcto: primero clítico átono como me/te/se/nos/ se + lo/la/los/las/le/les)
    for a in CLITICS:
        for b in CLITICS:
            combos.add(a + b)
    # opcional: podrías añadir triples si lo necesitas
    return sorted(combos, key=lambda x: -len(x))  # orden largo -> corto

CLITIC_COMBOS = generate_clitic_combos()

def analyze_text(nlp, text):
    """Analiza text y devuelve el primer word analysis (upos, feats, lemma) si existe."""
    doc = nlp(text)
    # si hay oraciones y words:
    if doc.sentences and doc.sentences[0].words:
        w = doc.sentences[0].words[0]
        return {'text': w.text, 'upos': w.upos, 'feats': w.feats or '', 'lemma': w.lemma}
    return None

def is_pronoun_like(nlp, token_text):
    a = analyze_text(nlp, token_text)
    return a is not None and (a['upos'] == 'PRON' or a['upos'] == 'DET')  # DET para artículos si aplicara

def try_split_token(nlp, token_text):
    """
    Devuelve lista de sub-tokens si decide dividir, 
    o [token_text] si no dividir.
    """
    token_text_orig = token_text
    token_text_lower = token_text_orig.lower()

    # 1) analiza token completo
    full = analyze_text(nlp, token_text_orig)
    # Si Stanza lo considera verbo 3ª pers. indicativo -> probablemente no es enclítico
    if full and full['upos'] == 'VERB' and 'Person=3' in full['feats'] and 'Mood=Imp' not in (full['feats'] or ''):
        return [token_text_orig]  # no dividimos

    # 2) intentamos splits por combinaciones de clíticos (de más largo a más corto)
    for combo in CLITIC_COMBOS:
        if token_text_lower.endswith(combo):
            left = token_text_orig[:len(token_text_orig)-len(combo)]
            right = token_text_orig[len(token_text_orig)-len(combo):]
            if not left: 
                continue

            # valida que cada pieza derecha se componga en pronombres conocidos (por si combo es raro)
            # descomponer right en secuencias de clíticos (intento simple: probar particion en 1..n)
            # para simplicidad, si combo está en la lista CLITICS o es concatenación válida, lo aceptamos provisionalmente
            # comprobación morfológica:
            left_a = analyze_text(nlp, left)
            right_a = analyze_text(nlp, right)

            # Conditions to accept split:
            # - left exists and is VERB
            # - right exists and is PRON (o sequence of PRONs)  (aquí cheque simple)
            if left_a and left_a['upos'] == 'VERB' and right_a and right_a['upos'] == 'PRON':
                # Heurística adicional: el verbo izquierdo debe mostrarse como imperativo o 2ª persona
                feats = (left_a['feats'] or '')
                if 'Mood=Imp' in feats or 'Person=2' in feats or 'VerbForm=Fin' in feats:
                    return [left, right]
                # otra heurística: si la forma original NO era verbo 3ª persona (evitamos 'comete' -> 'come+te')
                if not (full and full['upos']=='VERB' and 'Person=3' in full['feats']):
                    return [left, right]
                # otherwise continue searching combos

    # si no encontramos nada aceptable
    return [token_text_orig]


# Ejemplo de uso
texts = ["Él comete errores.", "Vete al norte.", "Cómete la manzana.", "comete", "dinos"]
for s in texts:
    print("===", s)
    # tokeniza rápido por espacio (ejemplo); en uso real recorre doc.sentences.tokens
    for tok in s.split():
        subtoks = try_split_token(nlp, tok)
        print(tok, "->", subtoks)
    print()


=== Él comete errores.
Él -> ['Él']
comete -> ['comete']
errores. -> ['errores.']

=== Vete al norte.
Vete -> ['Ve', 'te']
al -> ['al']
norte. -> ['norte.']

=== Cómete la manzana.
Cómete -> ['Cómete']
la -> ['la']
manzana. -> ['manzana.']

=== comete
comete -> ['comete']

=== dinos
dinos -> ['dinos']



In [43]:
nlp = stanza.Pipeline(
    "es",
    processors="tokenize,mwt,pos,lemma",
    tokenize_no_ssplit=False
)

doc = nlp("Él comete errores. Vete al norte.")
for sent in doc.sentences:
    for w in sent.words:
        print(w.text, w.lemma, w.upos, w.feats)

Él él PRON Case=Acc,Nom|Gender=Masc|Number=Sing|Person=3|PronType=Prs
comete cometer VERB Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
errores error NOUN Gender=Masc|Number=Plur
. . PUNCT PunctType=Peri
Vete vete PROPN None
a a ADP None
el el DET Definite=Def|Gender=Masc|Number=Sing|PronType=Art
norte norte NOUN Gender=Masc|Number=Sing
. . PUNCT PunctType=Peri


### 1.3.5 Stanza (minúsculas)

In [32]:
nlp = stanza.Pipeline('es', verbose=False)
test5 = []
for text in corpus: test5.append(nlp(text.lower()))
for doc in test5:
    for sentence in doc.sentences:
        for word in sentence.words:
            print(word.text, word.pos)
    print()

el DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

el DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

el DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
a ADP
el DET
taller NOUN
. PUNCT

la DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
a ADP
el DET
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

a ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

el DET
hermano NOUN
de ADP
maría NOUN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

en ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

el DE

Aquí cambia ligeramente: "di" lo marca bien como verbo, mientras que "dímelo" lo marca como sustantivo, y lo demás se mantiene igual. Aunque sea parecido a Spacy, este parece menos consistente con verbos con enclíticos. Ahora probamos con el último modelo, Flair.

### 1.3.6 Flair

In [33]:
# Cargar modelo POS (multilingual)
tagger = SequenceTagger.load("pos-multi")

for text in corpus:
    sentence = Sentence(text.lower())
    tagger.predict(sentence)
    for token in sentence:
        print(token.text, token.get_labels()[0].value)
    print()

2025-12-28 16:31:26,353 SequenceTagger predicts: Dictionary with 17 tags: NOUN, PUNCT, ADP, VERB, ADJ, DET, PROPN, ADV, PRON, AUX, CCONJ, NUM, SCONJ, PART, X, SYM, INTJ
el DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

el DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

el DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que CCONJ
lo PRON
llevé VERB
al DET
taller NOUN
. PUNCT

la DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADJ
. PUNCT

entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

a ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

el DET
hermano NOUN
de ADP
maría NOU

En este modelo vemos más inconsistencias todavía. "di" lo marca como ADP, y "lo" al separar "dímelo" lo marca como verbo. No mejora en ningún aspecto a los anteriores. <br>
Para mejorar este comportamiento vamos a hacer un preprocesamiento personalizado, el cual tratará de separar los enclíticos, pero para ello tenemos que ver qué verbos tienen enclíticos y cuáles no.

# 2 Preprocesamiento personalizado
Tras varias pruebas pensamos que el Spacy con dep news (texto en minúsculas) es el más consistente, ya que solo falla en los enclíticos. Para mejorar con los enclíticos podemos hacer nuestro propio preprocesamiento al texto para que así lo etiquete como debe.  
En primer lugar vamos a probar a utilizar un stemmer para sacar la raíz de las palabras, de esta forma vamos a ver si las etiquetas se mantienen como antes y, además, resuelve los verbos con enclíticos y los verbos con falsos enclíticos (i.e. verbos como "comete", "viste", etc.). Idealmente queremos que se lematicen los verbos sin enclíticos y los que sí los tienen los deje igual para luego separarlos. 
Si esto no funciona como esperamos entonces cogeremos la lista de pronombres que pueden ir junto a los verbos y los ponemos en orden (i.e. nunca se tiene "lasme", siempre "melas", como en "dámelas"). Pero esto solo no es suficiente, puesto que hay verbos que tienen "me", "te" u otros. Pero comete es diferente de cómete, de comer, por lo que necesitamos una lista con verbos a los que no les tenemos que aplicar este preprocesamiento. 

## 2.1 Imports
Primero definimos los imports que usaremos en esta parte, al igual que hicimos antes.

In [35]:
from nltk.stem import SnowballStemmer, LancasterStemmer
from nltk.tokenize import word_tokenize

## 2.2 Funciones auxiliares
Para no repetir código, vamos a definir una o varias funciones auxiliares.

In [36]:
def pos_tag(corpus: list[str] = []):
    nlp = spacy.load("es_dep_news_trf")
    docs = []
    for text in corpus: docs.append(nlp(text.lower()))
    return docs 

In [37]:
def compare_tags(docs: list, stems: list, compare: list):
    for doc, cmp, stem_phrase in zip(docs, compare, stems):
        for doc_token, cmp_token, stem in zip(doc, cmp, stem_phrase):
            if doc_token.pos_ != cmp_token.pos_:
                print('-'*40, '\n', 
                      doc_token.text, stem, doc_token.pos_, '|', cmp_token.text, cmp_token.pos_,
                      '\n', '-'*40)
            print(doc_token.text, stem, doc_token.pos_, '|', cmp_token.text, cmp_token.pos_)
        print() 

In [38]:
def print_tags(docs: list = [], stems: list = [], compare: list | None = None):
    if compare:
        compare_tags(docs, stems, compare)
    else:
        for doc in docs:
            for token, stem in zip(doc, stems):
                print(token.text, stem, token.pos_)
            print()    

## 2.3 Pruebas
Vamos ahora a probar los modelos para arreglar el fallo con los enclíticos.

### 2.3.1 LancasterStemmer

In [39]:
tokens_list = [word_tokenize(text.lower(), language="spanish") for text in corpus]
stemmer = LancasterStemmer()
stems = []
for token_phrase in tokens_list:
    stems.append([stemmer.stem(token) for token in token_phrase])

docs = pos_tag(corpus)
print_tags(docs, stems, test3)

el el DET | el DET
gato gato NOUN | gato NOUN
come com VERB | come VERB
pescado pescado NOUN | pescado NOUN
. . PUNCT | . PUNCT

el el DET | el DET
equipo equipo NOUN | equipo NOUN
de de ADP | de ADP
fútbol fútbol NOUN | fútbol NOUN
jugó jugó VERB | jugó VERB
su su DET | su DET
partido partido NOUN | partido NOUN
. . PUNCT | . PUNCT
este est PRON | este PRON
ganó ganó VERB | ganó VERB
con con ADP | con ADP
facilidad facilidad NOUN | facilidad NOUN
. . PUNCT | . PUNCT

el el DET | el DET
coche coch NOUN | coche NOUN
rojo rojo ADJ | rojo ADJ
se se PRON | se PRON
estropeó estropeó VERB | estropeó VERB
, , PUNCT | , PUNCT
así así ADV | así ADV
que que SCONJ | que SCONJ
lo lo PRON | lo PRON
llevé llevé VERB | llevé VERB
al al ADP | al ADP
taller tal NOUN | taller NOUN
. . PUNCT | . PUNCT

la la DET | la DET
presidenta president NOUN | presidenta NOUN
y y CCONJ | y CCONJ
el el DET | el DET
director direct NOUN | director NOUN
se se PRON | se PRON
reunieron reunieron VERB | reunieron VERB
; ;

Vemos que evalúa exactamente igual que con el texto. Además, algunos verbos con enclíticos los lematiza de forma que se pierde el pronombre (e.g. muéstrame lo lematiza como muéstram), por lo que no nos sirve este modelo.

### 2.3.2 SnowballStemmer

In [50]:
tokens_list = [word_tokenize(text.lower(), language="spanish") for text in corpus]
stemmer = SnowballStemmer("spanish")
stems = []
for token_phrase in tokens_list:
    stems.append([stemmer.stem(token) for token in token_phrase])

docs = pos_tag(corpus)
print_tags(docs, stems, test3)

el el DET | el DET
gato gat NOUN | gato NOUN
come com VERB | come VERB
pescado pesc NOUN | pescado NOUN
. . PUNCT | . PUNCT

el el DET | el DET
equipo equip NOUN | equipo NOUN
de de ADP | de ADP
fútbol futbol NOUN | fútbol NOUN
jugó jug VERB | jugó VERB
su su DET | su DET
partido part NOUN | partido NOUN
. . PUNCT | . PUNCT
este este PRON | este PRON
ganó gan VERB | ganó VERB
con con ADP | con ADP
facilidad facil NOUN | facilidad NOUN
. . PUNCT | . PUNCT

el el DET | el DET
coche coch NOUN | coche NOUN
rojo roj ADJ | rojo ADJ
se se PRON | se PRON
estropeó estrope VERB | estropeó VERB
, , PUNCT | , PUNCT
así asi ADV | así ADV
que que SCONJ | que SCONJ
lo lo PRON | lo PRON
llevé llev VERB | llevé VERB
al al ADP | al ADP
taller tall NOUN | taller NOUN
. . PUNCT | . PUNCT

la la DET | la DET
presidenta president NOUN | presidenta NOUN
y y CCONJ | y CCONJ
el el DET | el DET
director director NOUN | director NOUN
se se PRON | se PRON
reunieron reun VERB | reunieron VERB
; ; PUNCT | ; PUNCT
e

Este caso funciona igual que el anterior.

In [ ]:
# Lista de clíticos españoles
CLITICOS = ["me","te","se","lo","la","los","las","le","les","nos","os"]

# Orden típico permitido en español
# (simplificado pero suficientemente bueno para casi todos los casos)
ORDEN_CLITICOS = [
    ["me","lo"], ["me","la"], ["me","los"], ["me","las"],
    ["te","lo"], ["te","la"], ["te","los"], ["te","las"],
    ["se","lo"], ["se","la"], ["se","los"], ["se","las"],
    ["nos","lo"], ["nos","la"], ["nos","los"], ["nos","las"],
    ["os","lo"], ["os","la"], ["os","los"], ["os","las"],
]

# Generar patrones de clíticos pegados (ej: "melo", "telo", "selo")
PATRONES = sorted(
    [ "".join(p) for p in ORDEN_CLITICOS ] + CLITICOS,
    key=len, reverse=True
)

VERBOS = [
    "comete",
    "somete",
    "arremete",
    "promete",
    "remete",
    "entromete",
    "admite",
    "emite",
    "omite",
    "remite",
    "permite",
    "transmite",
    "dimite",
    "limite",
    "intermite",
    "retransmite",
    "siente",
    "consiente",
    "presiente",
    "resiente",
    "asiente",
    "disiente",
    "resume",
    "asume",
    "presume",
    "consume",
    "subsume",
    "come",
    "vale",
    'comeríamos'
] # Aún faltan muchísimos, igual mejor guardarlo como txt

In [ ]:
def separar_cliticos(token, lemma):
    """
    Separa clíticos de una forma verbal como 'dímelo', 'háblalo', 'cómetelo', etc.
    """
    t = token.lower()

    # Buscar si termina en un clítico válido (simple o doble)
    encontrados = []
    resto = t

    for patron in PATRONES:
        if resto.endswith(patron):
            encontrados.append(patron)          # ej: "melo"
            resto = resto[: -len(patron)]
            break

    # Si no encontró nada → no hay clíticos pegados
    if not encontrados:
        return [token]  

    # Convertir el verbo a su forma verbal (usar lemma como base)
    raiz = lemma

    # Normalizar la raíz para modo imperativo (caso típico)
    # Esto no es perfecto, pero funciona para mayoría:
    if raiz.endswith("r"):      # comer → come
        raiz = raiz[:-1]
    elif raiz.endswith("irse"): # "irse" → "ir" pero con reflexivos es complejo
        raiz = raiz[:-3]

    resultado = [raiz]

    # Ahora separar los clíticos (simple o doble)
    cl = encontrados[0]

    # Si es doble clítico ("melo", "selo", etc.)
    for c in CLITICOS:
        if cl.startswith(c):
            resto2 = cl[len(c):]
            if c in CLITICOS:
                resultado.append(c)
            if resto2 in CLITICOS:
                resultado.append(resto2)
            break
    
    return resultado

# 3 Dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset("PleIAs/Spanish-PD-Books", split="train")
print(dataset)


c:\Users\Jorge\Documents\Proyectos de programación\PLN\Desambiguador-correferencias\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Repo card metadata block was not found. Setting CardData to empty.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating train split: 2585 examples [00:04, 590.37 examples/s]Failed to read file 'C:\Users\Jorge\.cache\huggingface\hub\datasets--PleIAs--Spanish-PD-Books\snapshots\001eaf13681f483069361dd82195ce279e12ed63\spanish_pd_100.parquet' with error CastError: Couldn't cast
directory: string
identifier: string
...1: int64
creator: string


DatasetGenerationError: An error occurred while generating the dataset